In [2]:
# importing packages
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,OrdinalEncoder,StandardScaler
from collections import defaultdict
labelEncoder_dict = defaultdict(LabelEncoder)

# from sklearn.preprocessing import StandardScaler

In [3]:
# Loading file data
df = pd.read_csv("survey_results_public.csv")
X = pd.DataFrame()

/opt/homebrew/Caskroom/miniforge/base/envs/stack-overflow-developer-survey-analysis/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (8,12,13,14,15,16,50,51,52,53,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Data Cleaning

In [4]:
X['OpenSource'] = df['OpenSource'].eq('Yes').mul(1)
X['Hobby'] = df['Hobby'].eq('Yes').mul(1)
X['Student'] = df['Student'].str.contains('Yes').mul(1)


In [5]:

# Categorical Encoding Columns
# CategoricalColumns = ['Country','Employment',,'UndergradMajor',]
# for col in CategoricalColumns:
#     X[col] = labelEncoder_dict[col].fit_transform(df[col])

In [6]:
# YearsCoding
YearCodingMap = {
    '3-5 years':4,
    '30 or more years':30, 
    '24-26 years':25, 
    '18-20 years':19,
    '6-8 years':7, 
    '9-11 years':10, 
    '0-2 years':1,
    '15-17 years':16,
    '12-14 years':13, 
    '21-23 years':22, 
    '27-29 years':28,
}
X['YearsCoding'] = df['YearsCoding'].replace(YearCodingMap)
X['YearsCoding'].fillna(X['YearsCoding'].mean(),inplace=True)

In [7]:
companySizeMap = {
    '20 to 99 employees':60,
    '10,000 or more employees':10000,
    '100 to 499 employees':300, 
    '10 to 19 employees':15,
    '500 to 999 employees':750, 
    '1,000 to 4,999 employees':3000,
    '5,000 to 9,999 employees':7500, 
    'Fewer than 10 employees':10
}
X['CompanySize'] = df['CompanySize'].replace(companySizeMap)
X.dropna(subset=["CompanySize"],inplace=True) #Droping NaN values

In [8]:
# Formal Education
dummy1 = pd.get_dummies(df['FormalEducation'], drop_first=True) 
dummy1.drop(['I never completed any formal education','Primary/elementary school','Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)','Some college/university study without earning a degree'],axis=1,inplace=True)
X.join(dummy1)

,OpenSource,Hobby,Student,YearsCoding,CompanySize,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Master’s degree (MA, MS, M.Eng., MBA, etc.)","Other doctoral degree (Ph.D, Ed.D., etc.)","Professional degree (JD, MD, etc.)"
0,0,1,0,4.0,60.0,1,0,0,0
1,1,1,0,30.0,10000.0,1,0,0,0
2,1,1,0,25.0,60.0,0,0,0,0
3,0,0,0,19.0,300.0,1,0,0,0
4,0,1,1,7.0,10000.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
71526,0,1,0,7.0,10.0,0,0,0,0
71527,0,1,1,4.0,750.0,1,0,0,0
71528,0,1,0,10.0,10.0,1,0,0,0
71529,1,1,0,7.0,10000.0,1,0,0,0


In [9]:
employedMap = {
    'Employed part-time':1, 
    'Employed full-time':1,
    'Independent contractor, freelancer, or self-employed':1,
    'Not employed, and not looking for work':0,
    'Not employed, but looking for work':0,
    'Retired':0
}
df['Employment'].unique()

array(['Employed part-time', 'Employed full-time',
       'Independent contractor, freelancer, or self-employed',
       'Not employed, and not looking for work',
       'Not employed, but looking for work', nan, 'Retired'], dtype=object)

In [10]:
# AssessJob and Benefits Added
df.fillna(df.iloc[:,17:38].mean(),inplace=True)
for col in df.iloc[:,17:38].columns:
    X[col] = df[col]



In [11]:
df['JobSatisfaction'].unique()

array(['Extremely satisfied', 'Moderately dissatisfied',
       'Moderately satisfied', 'Neither satisfied nor dissatisfied',
       'Slightly satisfied', nan, 'Slightly dissatisfied',
       'Extremely dissatisfied'], dtype=object)

In [119]:
# JobSatisfaction Mapping
SatisfactionMapping = {
    'Extremely satisfied':6,
    'Moderately satisfied':5, 
    'Slightly satisfied':4,
    'Neither satisfied nor dissatisfied':3,
    'Moderately dissatisfied':2,
    'Slightly dissatisfied':1,
    'Extremely dissatisfied':0
}

df['JobSatisfaction'].replace(SatisfactionMapping,inplace=True)
df.fillna({'JobSatisfaction':3},inplace=True)
X['JobSatisfaction'] = df['JobSatisfaction']

# CareerSatisfaction
df['CareerSatisfaction'].replace(SatisfactionMapping,inplace=True)
df.fillna({'CareerSatisfaction':3},inplace=True)
X['CareerSatisfaction'] = df['CareerSatisfaction']



In [138]:
# HackathonReasons
X['HackathonParticipated'] = df['HackathonReasons'].notna()*1

# ConvertedSalary
X['ConvertedSalary'] = df['ConvertedSalary']
X.dropna(subset=["ConvertedSalary"],inplace=True) #Droping NaN values


### One Hot Encoding

In [121]:
def CustomOneHotEncoding(data,X):
    temp = data.str.split(';', expand=True)
    new_columns = pd.unique(temp.values.ravel())
    for col in new_columns:
        X[col] = data.str.contains(col, regex=False).fillna(False)*1

In [122]:
# LanguageWorkedWith
CustomOneHotEncoding(df['LanguageWorkedWith'],X)


CustomOneHotEncoding(df['DevType'],X)

CustomOneHotEncoding(df['DatabaseWorkedWith'],X)

CustomOneHotEncoding(df['PlatformWorkedWith'],X)

CustomOneHotEncoding(df['FrameworkWorkedWith'],X)

CustomOneHotEncoding(df['IDE'],X)

# Methodology
CustomOneHotEncoding(df['Methodology'],X)

# RaceEthnicity
CustomOneHotEncoding(df['RaceEthnicity'],X)

# CustomOneHotEncoding(df['Methodology'],X)


/var/folders/3h/smy7r7qj3wlgbhwx0plqyrph0000gn/T/ipykernel_79889/3727580376.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  X[col] = data.str.contains(col, regex=False).fillna(False)*1


In [123]:
# CheckInCode
CheckInCodeMapping = {
    'Multiple times per day':730, 
    'A few times per week':156, 
    'Weekly or a few times per month':52, 
    'Never':0,
    'Less than once per month':12, 
    'Once a day':365
}

X['CheckInCode'] = df['CheckInCode'].replace(CheckInCodeMapping)
X['CheckInCode'].fillna(X['CheckInCode'].mean(),inplace=True)

/var/folders/3h/smy7r7qj3wlgbhwx0plqyrph0000gn/T/ipykernel_79889/746842630.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  X['CheckInCode'] = df['CheckInCode'].replace(CheckInCodeMapping)


In [124]:
AgeMapping = {
    '25 - 34 years old':29.5, 
    '35 - 44 years old':39.5, 
    '18 - 24 years old':21,
    '45 - 54 years old':49.5, 
    '55 - 64 years old':59.5, 
    'Under 18 years old':18,
    '65 years or older':65
}
X['Age'] = df['Age'].replace(AgeMapping)
X.dropna(subset=["Age"],inplace=True) #Droping NaN values

X['MilitaryUS'] = (df['MilitaryUS']=='Yes')*1
X['Dependents'] = (df['Dependents']=='Yes')*1
X['Gender'] = (df['Gender']=='Female')*1


/var/folders/3h/smy7r7qj3wlgbhwx0plqyrph0000gn/T/ipykernel_79889/1484029417.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  X['Age'] = df['Age'].replace(AgeMapping)


In [125]:
# Exercise
ExerciseFreqMap = {
    '3 - 4 times per week':((3+4)/2)*52,
    'Daily or almost every day':365, 
    "I don't typically exercise":0, 
    '1 - 2 times per week':52
}
X['Exercise'] = df['Exercise'].replace(ExerciseFreqMap)
X['Exercise'].fillna(X['Exercise'].mean(),inplace=True)

# HoursCompMap
HoursCompMap = {
    '9 - 12 hours':10.5, 
    '5 - 8 hours':6.5, 
    'Over 12 hours':12, 
    '1 - 4 hours':2.5,
    'Less than 1 hour':1
}
X['HoursComputer'] = df['HoursComputer'].replace(HoursCompMap)
X['HoursComputer'].fillna(X['HoursComputer'].mean(),inplace=True)


In [126]:
# HypotheticalTools1-5
HypoToolMap = {
    'Extremely interested':5, 
    'Very interested':4, 
    'Somewhat interested':3, 
    'A little bit interested':2,
    'Not at all interested':1
}
hypotheticalToolsList = ['HypotheticalTools1','HypotheticalTools2','HypotheticalTools3','HypotheticalTools4','HypotheticalTools5']
for col in hypotheticalToolsList:
    X[col] = df[col].replace(HypoToolMap)
    X[col].fillna(X[col].median(),inplace=True)


In [127]:
# EducationParents -> Higher Educated Parents
EducatedParentsMap = {
    "Bachelor’s degree (BA, BS, B.Eng., etc.)":1,
    'Some college/university study without earning a degree':0,
    'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)':0,
    "Master’s degree (MA, MS, M.Eng., MBA, etc.)":1,
    'Primary/elementary school':0, 
    'Associate degree':1,
    'They never completed any formal education':0,
    'Other doctoral degree (Ph.D, Ed.D., etc.)':1,
    'Professional degree (JD, MD, etc.)':1
}
X['ParentsWithHighEducation'] = df['EducationParents'].replace(EducatedParentsMap)
X.dropna(subset=["ParentsWithHighEducation"],inplace=True) #Droping NaN values


/opt/homebrew/Caskroom/miniforge/base/envs/stack-overflow-developer-survey-analysis/lib/python3.9/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['NoneType', 'float', 'str']. An error will be raised in 1.2.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/stack-overflow-developer-survey-analysis/lib/python3.9/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['NoneType', 'float', 'str']. An error will be raised in 1.2.
  warnings.warn(


## Modeling and Predictions
### Using Regression

In [129]:
# loading additional packages
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error,accuracy_score

In [157]:
# Partitioning data into test and train 
Y_columns = ['JobSatisfaction', 'CareerSatisfaction','ConvertedSalary']

df_Y = X_std[Y_columns]
df_X = X_std.loc[:, ~X_std.columns.isin(Y_columns)]
df_X_train, df_X_test, df_Y_train, df_Y_test = train_test_split(df_X,df_Y,test_size=0.2, shuffle=True, random_state=1)


/var/folders/3h/smy7r7qj3wlgbhwx0plqyrph0000gn/T/ipykernel_79889/273901148.py:4: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  df_Y = X_std[Y_columns]


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [152]:
# Linear Regression
reg = linear_model.LinearRegression()
reg.fit(df_X_train,df_Y_train)
df_Y_test_pred = reg.predict(df_X_test)
df_Y_train_pred = reg.predict(df_X_train)

# i) training error
print("training error", mean_squared_error(df_Y_train, df_Y_train_pred))
# iI) testing error
print("testing error", mean_squared_error(df_Y_test, df_Y_test_pred))


/opt/homebrew/Caskroom/miniforge/base/envs/stack-overflow-developer-survey-analysis/lib/python3.9/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['NoneType', 'float', 'str']. An error will be raised in 1.2.
  warnings.warn(


training error 13130087251.098755
testing error 14172190866.133085


/opt/homebrew/Caskroom/miniforge/base/envs/stack-overflow-developer-survey-analysis/lib/python3.9/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['NoneType', 'float', 'str']. An error will be raised in 1.2.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/stack-overflow-developer-survey-analysis/lib/python3.9/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['NoneType', 'float', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [ ]:
# Ridge regression with hyperparamter values alpha from [0.0, 1e-8, 1e-5, 0.1, 1, 10]
alphas = [0.0, 1e-8, 1e-5, 0.1, 1, 10]
alphaErrMap = {}
for alpha in alphas:
    reg = linear_model.Ridge(alpha=alpha)
    reg.fit(df_X_train,df_Y_train)
    df_Y_test_pred = reg.predict(df_X_test)
    df_Y_train_pred = reg.predict(df_X_train)
    training_error = mean_squared_error(df_Y_train, df_Y_train_pred)
    testing_error = mean_squared_error(df_Y_test, df_Y_test_pred)
    # i) training error
    print("training error",alpha, training_error)
    # iI) testing error
    print("testing error",alpha, testing_error)
    alphaErrMap[alpha] = testing_error

optimal_alpha = min(alphaErrMap, key=alphaErrMap.get)
print("optimal_alpha",optimal_alpha)

In [ ]:
# Ridge regression with hyperparamter values alpha from [0.0, 1e-8, 1e-5, 0.1, 1, 10]
alphas = [0.0, 1e-8, 1e-5, 0.1, 1, 10]
alphaErrMap = {}
for alpha in alphas:
    reg = linear_model.Lasso(alpha=alpha)
    reg.fit(df_X_train,df_Y_train)
    df_Y_test_pred = reg.predict(df_X_test)
    df_Y_train_pred = reg.predict(df_X_train)
    training_error = mean_squared_error(df_Y_train, df_Y_train_pred)
    testing_error = mean_squared_error(df_Y_test, df_Y_test_pred)
    # i) training error
    print("training error",alpha, training_error)
    # iI) testing error
    print("testing error",alpha, testing_error)
    alphaErrMap[alpha] = testing_error

optimal_alpha = min(alphaErrMap, key=alphaErrMap.get)
print("optimal_alpha",optimal_alpha)